In [8]:
import sys
sys.path.insert(0,'/home/ghcp/Documentos/CINVESTAV/SEMESTRE_1/scripts')
sys.path.insert(0,'/home/ghcp/Documentos/CINVESTAV/ANALISYS_B0tomumuKstar/NOTEBOOKS')
from plot_tools import create_axes_for_pulls, plot_model 
import zfit
import plot_tools 
import numpy as np
import customPDFs
import mass_models
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
import seaborn as sns
import mplhep as hep
from zfit import z
import xgboost as xgb
import pandas as pd
import os
pd.set_option('display.max_columns', None)
import uproot3 as uproot
import os, sys
import common_tools
from scipy.stats import gaussian_kde

from zfit import z
from zfit import pdf
import common_tools
from zfit.pdf import Chebyshev  # Correct import path
from scipy.stats import gaussian_kde

In [9]:
model = xgb.XGBClassifier()
model.load_model("/home/ghcp/Documentos/CINVESTAV/ANALISYS_B0tomumuKstar/XGBoost/myEarly2023Analizer-main/backgroundReduction/XGBoost/hyperOptimization/hyperOptimization/results_ho/auc/best_model.json")
features = ['cosalfaB', 'Bdl', 'Pi2pt', 'Pi1pt', 'muon_dca', 'Bpt', 'Bprob', 'Jprob']



def process_xgboost_cut(filepath, tree_name="treeBd", chunk_size=9_000_000, fom_cut=0.9933, output_path="output_filtered.root"):
    results = []
    with uproot.open(filepath) as file:
        tree = file[tree_name]
        num_entries = tree.numentries   # ✅ en uproot3 es numentries
        
        for start in range(0, num_entries, chunk_size):
            stop = min(start + chunk_size, num_entries)

            # Cargar chunk como DataFrame
            chunk = tree.arrays(
                outputtype=pd.DataFrame, 
                entrystart=start, 
                entrystop=stop
            )
            
            # Verificar features
            missing = [f for f in features if f not in chunk.columns]
            if missing:
                raise ValueError(f"Faltan columnas en el árbol: {missing}")
            
            # Convertir a float32 para ahorrar RAM
            X_new = chunk[features].astype("float32")
            
            # Predicciones
            y_pred_proba = model.predict_proba(X_new)[:, 1]
            y_pred = model.predict(X_new)
            
            # Filtro inicial
            mask = (y_pred == 1) & (y_pred_proba >= 0.80)
            filtered_chunk = chunk[mask].copy()
            filtered_chunk["score"] = y_pred_proba[mask]
            filtered_chunk["prediction"] = y_pred[mask]
            
            # Corte fuerte
            filtered_chunk = filtered_chunk[filtered_chunk["score"] >= fom_cut]
            
            # Guardar resultados parciales
            if not filtered_chunk.empty:
                results.append(filtered_chunk)
    
    if results:
        df_result = pd.concat(results, ignore_index=True)
        print("Eventos finales:", filepath, len(df_result))

        # Convertir DataFrame → diccionario numpy para uproot3
        branch_dict = {col: df_result[col].values for col in df_result.columns}

        # Guardar en archivo ROOT con uproot3
        # with uproot.recreate("/home/ghcp/Documentos/CINVESTAV/ANALISYS_B0tomumuKstar/CODES/" + output_path) as f:
        with uproot.recreate("/home/ghcp/Documentos/CINVESTAV/ANALISYS_B0tomumuKstar/angular/codes/" + output_path) as f:

            
            f["treeBd"] = uproot.newtree({col: "float32" for col in df_result.columns})
            f["treeBd"].extend(branch_dict)

        print("✅ Archivo ROOT guardado con uproot3 en:", output_path)
        return df_result
    else:
        print("No se encontraron eventos que pasen los cortes.")
        return pd.DataFrame()
    




def process_xgboost_window_resonant(filepath, tree_name="treeBd", chunk_size=1_000_000, fom_cut=0.9955, chanel="jpsi"):
    results = []
    if chanel=="jpsi":  
        q_2_min = 8.0
        q_2_max = 11.0
    else: 
        q_2_min = 12.5
        q_2_max = 15.0
    
    with uproot.open(filepath) as file:
        tree = file[tree_name]
        num_entries = tree.numentries
        
        for start in range(0, num_entries, chunk_size):
            stop = min(start + chunk_size, num_entries)

            # Cargar chunk como DataFrame
            chunk = tree.arrays(
                outputtype=pd.DataFrame, 
                entrystart=start, 
                entrystop=stop
            )
            
            # PRIMERO: Aplicar filtro de masa
            chunk["q_2"] = chunk["massJ"]**2

            mass_mask = (chunk["q_2"] > q_2_min) & (chunk["q_2"] < q_2_max)
            chunk_mass_filtered = chunk[mass_mask].copy()
            
            # Si no hay eventos después del corte de masa, saltar este chunk
            if chunk_mass_filtered.empty:
                continue
            
            # Verificar features en el chunk filtrado
            missing = [f for f in features if f not in chunk_mass_filtered.columns]
            if missing:
                raise ValueError(f"Faltan columnas en el árbol: {missing}")
            
            # Convertir a float32 para ahorrar RAM
            X_filtered = chunk_mass_filtered[features].astype("float32")
            
            # SEGUNDO: Aplicar XGBoost solo a eventos dentro de la ventana de masa
            y_pred_proba = model.predict_proba(X_filtered)[:, 1]
            
            # Aplicar SOLO el corte fuerte de FOM
            fom_mask = y_pred_proba >= fom_cut
            final_chunk = chunk_mass_filtered[fom_mask].copy()
            final_chunk["score"] = y_pred_proba[fom_mask]
            
            # Guardar resultados parciales
            if not final_chunk.empty:
                results.append(final_chunk)

    if results:
        df_result = pd.concat(results, ignore_index=True)
        branch_dict = {col: df_result[col].values for col in df_result.columns}
        print(f"Eventos finales en {filepath}: {len(df_result)}")
                # Guardar en archivo ROOT con uproot3
        output_path ="data_for_fit_reso_" + chanel
        with uproot.recreate("/home/ghcp/Documentos/CINVESTAV/ANALISYS_B0tomumuKstar/angular/codes/" + output_path+".root") as f:
            f[tree_name] = uproot.newtree({col: "float32" for col in df_result.columns})
            f[tree_name].extend(branch_dict)

        print("✅ Archivo ROOT guardado con uproot3 en:", output_path)
        return df_result
    else:
        print("No se encontraron eventos que pasen los cortes.")
        return pd.DataFrame()




In [10]:
path_data_v0     = "/home/ghcp/Documentos/CINVESTAV/ANALISYS_B0tomumuKstar/angular/codes/AntiRadVeto_Data_2022_Era1_v0.root"
#path_MC_Jpsi_v0  = "/home/ghcp/Documentos/CINVESTAV/ANALISYS_B0tomumuKstar/angular/codes/AntiRadVeto_MC_Jpsi_2022_Era1_v0.root"
#path_MC_psi2s_v0 = "/home/ghcp/Documentos/CINVESTAV/ANALISYS_B0tomumuKstar/angular/codes/AntiRadVeto_MC_Psi2s_2022_Era1_v0.root"
# path_MC_nores_v0 = "/home/ghcp/Documentos/CINVESTAV/ANALISYS_B0tomumuKstar/angular/codes/AntiRadVeto_MC_NoRes_2022_Era1_v0.root"

# output_path_data_v0  = "AntiRadVeto_Data_2022_Era1_v0_XGBoost_fom_cut.root"
#output_path_Jpsi_v0  = "AntiRadVeto_MC_Jpsi_2022_Era1_v0_XGBoost_fom_cut.root"
#output_path_Psi2s_v0 = "AntiRadVeto_MC_Psi2s_2022_Era1_v0_XGBoost_fom_cut.root"
# output_path_NoRes_v0 = "AntiRadVeto_MC_NoRes_2022_Era1_v0_XGBoost_fom_cut.root"

# df_data_o     = process_xgboost_cut(filepath = path_data_v0,     output_path=output_path_data_v0) 
#df_MC_Jpsi_o  = process_xgboost_cut(filepath = path_MC_Jpsi_v0,  output_path=output_path_Jpsi_v0)
#df_MC_psi2s_o = process_xgboost_cut(filepath = path_MC_psi2s_v0, output_path=output_path_Psi2s_v0)
# df_MC_nores_o = process_xgboost_cut(filepath = path_MC_nores_v0, output_path=output_path_NoRes_v0)


#path_data_v1     = "/home/ghcp/Documentos/CINVESTAV/ANALISYS_B0tomumuKstar/angular/codes/AntiRadVeto_Data_2022_Era1_v1.root"
# path_MC_Jpsi_v1  = "/home/ghcp/Documentos/CINVESTAV/ANALISYS_B0tomumuKstar/CODES/AntiRadVeto_MC_Jpsi_2022_Era1_v1.root"
# path_MC_psi2s_v1 = "/home/ghcp/Documentos/CINVESTAV/ANALISYS_B0tomumuKstar/CODES/AntiRadVeto_MC_Psi2s_2022_Era1_v1.root"
# path_MC_nores_v1 = "/home/ghcp/Documentos/CINVESTAV/ANALISYS_B0tomumuKstar/CODES/AntiRadVeto_MC_NoRes_2022_Era1_v1.root"

#output_path_data_v1  = "AntiRadVeto_Data_2022_Era1_v1_XGBoost_fom_cut.root"
# output_path_Jpsi_v1  = "AntiRadVeto_MC_Jpsi_2022_Era1_v1_XGBoost_fom_cut.root"
# output_path_Psi2s_v1 = "AntiRadVeto_MC_Psi2s_2022_Era1_v1_XGBoost_fom_cut.root"
# output_path_NoRes_v1 = "AntiRadVeto_MC_NoRes_2022_Era1_v1_XGBoost_fom_cut.root"

#df_data_o_v1  = process_xgboost_cut(filepath = path_data_v1,  output_path=output_path_data_v1)
# df_MC_Jpsi_o_v1  = process_xgboost_cut(filepath = path_MC_Jpsi_v1,  output_path=output_path_Jpsi_v1)
# df_MC_psi2s_o_v1 = process_xgboost_cut(filepath = path_MC_psi2s_v1, output_path=output_path_Psi2s_v1)
# df_MC_nores_o_v1 = process_xgboost_cut(filepath = path_MC_nores_v1, output_path=output_path_NoRes_v1)



# fileNoRes = uproot.open(path_MC_nores)
# fileJPsi  = uproot.open(path_MC_Jpsi)
# filePsi2S = uproot.open(path_MC_psi2s)

# treeJPsi=fileJPsi['treeBd']
# dfJPsi = treeJPsi.arrays(outputtype=pd.DataFrame)
#df_data_o     = process_xgboost_cut(path_data)



In [11]:
df_data_Jpsi =process_xgboost_window_resonant(filepath = path_data_v0 , chunk_size=9_000_000, chanel="jpsi")
df_data_psi2s =process_xgboost_window_resonant(filepath =path_data_v0, chunk_size=9_000_000, chanel="psi2s")


Eventos finales en /home/ghcp/Documentos/CINVESTAV/ANALISYS_B0tomumuKstar/angular/codes/AntiRadVeto_Data_2022_Era1_v0.root: 25567
✅ Archivo ROOT guardado con uproot3 en: data_for_fit_reso_jpsi
Eventos finales en /home/ghcp/Documentos/CINVESTAV/ANALISYS_B0tomumuKstar/angular/codes/AntiRadVeto_Data_2022_Era1_v0.root: 2579
✅ Archivo ROOT guardado con uproot3 en: data_for_fit_reso_psi2s
